<a href="https://colab.research.google.com/github/Fliptoss/ML_stuff/blob/main/Without_using_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]

In [ ]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [ ]:
df.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

In [ ]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head(2)

Pclass   Sex   Age  SibSp  Parch  Fare Embarked
331       1  male  45.5      0      0  28.5        S
733       2  male  23.0      0      0  13.0        S

In [ ]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

for missing values, we have to do SimpleImputer,
for sex and embarked, we have to perform OneHotEncoding

In [ ]:
## si for age and embarked
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

In [ ]:
X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])


X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [ ]:
X_train_age, X_train_embarked

(array([[45.5       ],
        [23.        ],
        [32.        ],
        [26.        ],
        [ 6.        ],
        [24.        ],
        [45.        ],
        [29.        ],
        [29.49884615],
        [29.49884615],
        [42.        ],
        [36.        ],
        [33.        ],
        [17.        ],
        [29.        ],
        [50.        ],
        [35.        ],
        [38.        ],
        [34.        ],
        [17.        ],
        [11.        ],
        [61.        ],
        [30.        ],
        [ 7.        ],
        [63.        ],
        [20.        ],
        [29.49884615],
        [29.        ],
        [36.        ],
        [29.49884615],
        [50.        ],
        [27.        ],
        [30.        ],
        [33.        ],
        [29.49884615],
        [29.49884615],
        [ 2.        ],
        [25.        ],
        [51.        ],
        [25.        ],
        [29.49884615],
        [29.49884615],
        [24.        ],
        [18

In [ ]:
## now we perform OneHotEncoding for both Sex and Embarked
ohe_sex = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_embarked = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [ ]:
X_train_embarked, X_train_sex

(array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]]),
 array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [1., 0.],
        [0., 1.]]))

In [ ]:
X_train.head() ## no effect on the main dataset. therefore we have to drop the unchanged and concat with the changed ones

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
331       1    male  45.5      0      0  28.5000        S
733       2    male  23.0      0      0  13.0000        S
382       3    male  32.0      0      0   7.9250        S
704       3    male  26.0      1      0   7.8542        S
813       3  female   6.0      4      2  31.2750        S

In [ ]:
X_train_rem = X_train.drop(columns=['Sex', 'Age', 'Embarked'])
X_test_rem = X_test.drop(columns=['Sex', 'Age', 'Embarked'])

In [ ]:
X_train_transformed = np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked), axis=1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked), axis=1)

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred = clf.predict(X_test_transformed)

In [ ]:
y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7821229050279329

In [ ]:
import pickle

In [ ]:
pickle.dump(ohe_sex, open('ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('ohe_embarked.pkl', 'wb'))
pickle.dump(clf, open('clf.pkl', 'wb'))